In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from kaggle_environments import evaluate, make, utils, register
import telebot
from telebot import types
from ALPHA_BETA import alphabeta_agent
from ALPHA_BETA_ON_TOR import alphabeta_on_tor_agent
from MCTS import MCTS_agent, is_win
from MCTS_ON_TOR import MCTS_on_tor_agent, make_turn, bad_move, is_win_tor, is_tie
from interpereters import interpreter, interpreter_tor
from utils import draw_board, renderer, print_board

In [6]:
true_env = make("connectx", debug=True)

specification = true_env.specification.copy()
specification["name"] = "ConnectXOnTor"
specification["title"] = "ConnectXOnTor"
specification["description"] = "ConnectX but On Tor"
specification['observation']['board']['default'] = [0] * 42
specification['configuration']['timeout']['default'] = 100

agents = {}
register(specification["name"], {
    "agents": agents,
    "interpreter": interpreter_tor,
    "renderer": renderer,
    "specification": specification,
})

In [8]:
make('connectx').sp

In [18]:
class TGBotForConnectX:
    def __init__(self) -> None:
        self.board_for_bot =  [0] * 42
        self.bot = telebot.TeleBot('your token')
        self.obs = None
        self.state = None
        self.agent = None
        self.env = None
        self.config = None
        self.mode = None
    def do_all(self):
        @self.bot.message_handler(commands=['start'])
        def start(message):

            self.bot.send_message(message.from_user.id, 'Это бот для игры Connect4 на торе. Каждый ход - это число от 1 до 7. \
                                   \nБоту может потребоваться до 2 минут, чтобы сделать ход',  parse_mode="Markdown")            
            self.bot.send_message(message.from_user.id, 'Если хочешь играть играть в обычном режиме, введи 1\nЕсли хочешь играть на торе, введи 2', 
                                   parse_mode="Markdown")            
            self.bot.register_next_step_handler(message, choose_mode)

        def incorrect_choose_mode(message):
            self.bot.send_message(message.from_user.id, 'Некорректный режим')
            self.bot.register_next_step_handler(message, choose_mode)

        def choose_mode(message):
            if message.text == '1':
                self.env = make('connectx', debug=True)
                self.env.specification['configuration']['timeout']['default'] = 100
                self.config = self.env.configuration
                self.mode = 1
                self.bot.send_message(message.from_user.id, 'Если хочешь ходить играть против MTCS, введи MTCS \nЕсли хочешь играть против alpha-beta, введи AB', 
                                   parse_mode="Markdown")            
                self.bot.register_next_step_handler(message, choose_bot)


            elif message.text == '2':
                self.env = make('ConnectXOnTor', debug=True)
                self.config = self.env.configuration
                self.mode = 2
                self.bot.register_next_step_handler(message, choose_bot)
                self.bot.send_message(message.from_user.id, 'Если хочешь ходить играть против MCTS, введи MCTS \nЕсли хочешь играть против alphf-beta, введи AB', 
                                   parse_mode="Markdown")            
            else:
                self.bot.register_next_step_handler(message, incorrect_choose_mode)

        def incorrect_choose_color(message):
            self.bot.send_message(message.from_user.id, 'Некорректный агент')
            self.bot.register_next_step_handler(message, choose_bot)

        def choose_bot(message):
            if message.text == 'MCTS':
                if self.mode == 1:
                    self.agent = MCTS_agent
                else:
                    self.agent = MCTS_on_tor_agent
                self.bot.send_message(message.from_user.id, 'Если хочешь ходить первым, введи 1 \nЕсли хочешь ходить вторым, введи 2')            
                self.bot.register_next_step_handler(message, choose_colour)

            elif message.text == 'AB':
                if self.mode == 1:
                    self.agent = alphabeta_agent
                else:
                    self.agent = alphabeta_on_tor_agent
                self.bot.send_message(message.from_user.id, 'Если хочешь ходить первым, введи 1 \nЕсли хочешь ходить вторым, введи 2')            
                self.bot.register_next_step_handler(message, choose_colour)
            else:
                self.bot.register_next_step_handler(message, incorrect_choose_mode)

        def incorrect_choose_color(message):
            self.bot.send_message(message.from_user.id, 'Некорректный ввод')
            self.bot.register_next_step_handler(message, choose_colour)

        def choose_colour(message):
            self.state = self.env.reset(2)[0]
            self.obs = self.state['observation']

            if message.text == '2':
                turn = self.agent(self.obs, self.config)

                self.state = self.env.step([turn, 200])[0]
                self.obs = self.state['observation']
                self.bot.send_message(message.from_user.id, draw_board(self.obs['board']),  parse_mode="Markdown")
                self.bot.register_next_step_handler(message, user_turn_second)
            elif message.text == '1':
                self.bot.send_message(message.from_user.id, draw_board(self.obs['board']),  parse_mode="Markdown")
                self.bot.send_message(message.from_user.id, 'сделай первый ход',  parse_mode="Markdown")
                self.bot.register_next_step_handler(message, user_turn_first)
            else: 
                self.bot.register_next_step_handler(message, incorrect_choose_color)

        def incorrect_turn_first(message):
            self.bot.send_message(message.from_user.id, 'Некорректный ход')
            self.bot.register_next_step_handler(message, user_turn_first)

        def incorrect_turn_second(message):
            self.bot.send_message(message.from_user.id, 'Некорректный ход')
            self.bot.register_next_step_handler(message, user_turn_second)

        def user_turn_second(message):
            available_moves = [move for move in range(7) if self.obs['board'][move] == 0]
            action_by_user = int(message.text) - 1

            if action_by_user not in available_moves:
                self.bot.register_next_step_handler(message, incorrect_turn_second)
            else:
                self.state = self.env.step([200, action_by_user])[0]
                self.obs = self.state['observation']
                self.bot.send_message(message.from_user.id, draw_board(self.obs['board']),  parse_mode="Markdown")
                if self.state['status'] == 'DONE':
                    self.bot.send_message(message.from_user.id, 'You win!')
                    self.bot.send_message(message.from_user.id, 'выбери номер хода, если хочешь сыграть еще')
                    self.bot.register_next_step_handler(message, choose_mode)
                else:
                    turn = self.agent(self.obs, self.config)
                    print(turn)
                    self.state = self.env.step([turn, 200])[0]
                    self.obs = self.state['observation']
                    self.bot.send_message(message.from_user.id, draw_board(self.obs['board']),  parse_mode="Markdown")
                    if self.state['status'] == 'DONE':
                        self.bot.send_message(message.from_user.id, 'You lose!')
                        self.bot.send_message(message.from_user.id, 'выбери номер хода, если хочешь сыграть еще')
                        self.bot.register_next_step_handler(message, choose_mode)
                    else:
                        self.bot.register_next_step_handler(message, user_turn_second)
                    
        def user_turn_first(message):

            available_moves = [move for move in range(7) if self.obs['board'][move] == 0]
            action_by_user = int(message.text) - 1

            if action_by_user not in available_moves:
                self.bot.register_next_step_handler(message, incorrect_turn_first)

            else:
                self.state = self.env.step([action_by_user, 200])[0]
                self.obs = self.state['observation']
                self.bot.send_message(message.from_user.id, draw_board(self.obs['board']),  parse_mode="Markdown")
                if self.state['status'] == 'DONE':
                    self.bot.send_message(message.from_user.id, 'You win!')
                    self.bot.send_message(message.from_user.id, 'выбери номер хода, если хочешь сыграть еще')
                    self.bot.register_next_step_handler(message, choose_mode)
                else:
                    turn = self.agent(self.obs, self.config)
                    self.state = self.env.step([200, turn])[0]
                    self.obs = self.state['observation']
                    self.bot.send_message(message.from_user.id, draw_board(self.obs['board']),  parse_mode="Markdown")
                    if self.state['status'] == 'DONE':
                        self.bot.send_message(message.from_user.id, 'You lose!')
                        self.bot.send_message(message.from_user.id, 'выбери номер хода, если хочешь сыграть еще')
                        self.bot.register_next_step_handler(message, choose_mode)
                    else:
                        self.bot.register_next_step_handler(message, user_turn_first)

        self.bot.polling()    

In [19]:
bot = TGBotForConnectX()
bot.do_all()